In [ ]:
!pip install mysql-connector-python
!pip install pandas mysql-connector-python
!pip install hdx-python-api



  Using cached mysql_connector_python-8.4.0-cp310-cp310-manylinux_2_17_x86_64.whl.metadata (2.0 kB)
Using cached mysql_connector_python-8.4.0-cp310-cp310-manylinux_2_17_x86_64.whl (19.4 MB)


In [31]:
import pandas as pd
from sqlalchemy import create_engine, text 
from sqlalchemy.engine import reflection
from sqlalchemy.exc import ProgrammingError
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
# Replace these values with your actual database information
config = {
    'user': 'soonho',
    'password': 'Yh2Wgfy2W4wYjvW36TTThzfDB',
    'host': 'fsp-raw-production.c5nzpjrxfrcj.us-east-1.rds.amazonaws.com',
    'database': 'fsp_data',
    'raise_on_warnings': True
    }

 # Create an SQLAlchemy engine for pandas to use
engine = create_engine(f'mysql+mysqlconnector://{config["user"]}:{config["password"]}@{config["host"]}/{config["database"]}')
Session = sessionmaker(bind=engine)
session = Session()


def truncate_existing_tables(tables):
    try:
        for table in tables:
            # Construct the TRUNCATE TABLE SQL query
            truncate_query = text(f"TRUNCATE TABLE {table}")
            
            try:
                # Execute the TRUNCATE TABLE query
                session.execute(truncate_query)
                session.commit()
                print(f"Table {table} truncated successfully.")
            except Exception as error:
                session.rollback()
                print(f"Error truncating table {table}: {error}")
    except Exception as error:
        print(f"Error connecting to the database: {error}")

def insert_Dataframe_To_DB(df, table_name):
    if not isinstance(df, pd.DataFrame):
        return False
    
    if not isinstance(table_name,str ):
        raise ValueError("table_name must be a string")   
        return False    


     # Create an SQLAlchemy engine for pandas to use
    #engine = create_engine(f'mysql+mysqlconnector://{config["user"]}:{config["password"]}@{config["host"]}/{config["database"]}')
    
    # Insert the DataFrame into MySQL, replacing the table if it exists
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    

    print(f'Data successfully inserted into {table_name}.')


In [32]:
# get the WFP data from HDX

from hdx.api.configuration import Configuration
from hdx.data.dataset import Dataset
import requests
import pandas as pd
from io import StringIO


if not Configuration._configuration:
    Configuration.create(hdx_site="prod", user_agent="Food Security Portal project", hdx_read_only=True)
        # Search for datasets
dataset = Dataset.read_from_hdx("wfp-food-prices-for-nepal")
resources = dataset.get_resources()
#print(resources)

csv_resource = next(resource for resource in resources if resource['format'] == 'CSV')
download_url = csv_resource['download_url']
print(download_url)

# Download the CSV file
csv_data = pd.read_csv(download_url)

# Print the first few rows of the CSV data
print(csv_data.head())

# REMOVE THE FIRST ROW, SINCE THERE IS UNNECESSARY INFORMATION

csv_data = csv_data.iloc[1:]

# ADD ISO3CODE SINCE THERE IS NO COUNTRYCODE ITSELF FROM THE CSV FILE

csv_data['ISO3Code'] = 'NPL'
df = pd.DataFrame();
df = csv_data


# truncate the exisiting tables 
tables = ['WFP_PRICE_DATA','WFP_PRICE_DATA_PROCESSED', 'WFP_PRICE_DATA_PROCESSED_MISSING REAL_REFERENCE', 'WFP_PRICE_DATA_PROCESSED_REAL_REFERENCE']

truncate_existing_tables(tables) 

insert_Dataframe_To_DB(df, 'WFP_PRICE_DATA')


https://data.humdata.org/dataset/0e8b1b84-8e55-4aec-bc45-0c7d2c92d32e/resource/10bae3d7-82bf-4caf-a216-5fb4a8ff6e43/download/wfp_food_prices_npl.csv
         date          admin1      admin2            market    latitude  \
0       #date      #adm1+name  #adm2+name  #loc+market+name    #geo+lat   
1  2001-04-15  Province No. 1    DHANKUTA          Dhankuta  26.9833333   
2  2001-04-15  Province No. 1        ILAM              Ilam     26.9103   
3  2001-04-15  Province No. 1       JHAPA             Jhapa   26.483333   
4  2001-04-15  Province No. 2    DHANUSHA          Dhanusha     26.7362   

    longitude            category      commodity        unit  \
0    #geo+lon          #item+type     #item+name  #item+unit   
1  87.3333333  cereals and tubers    Wheat flour          KG   
2     87.9375  cereals and tubers  Rice (coarse)          KG   
3       87.85  cereals and tubers    Wheat flour          KG   
4     85.9292  cereals and tubers  Rice (coarse)          KG   

          price

## Deleting datapoint where the price is 0

In [33]:
print("Number of rows:", df.shape[0])
print(df['price'].dtype)

df = df[(df['price'] != 0.0) & (df['priceflag'] == "actual")]
print("Number of rows:", df.shape[0])


Number of rows: 25253
object
Number of rows: 25253


## Calculating the annual percentage change for seasonality

In [34]:
percentage_change_df = pd.DataFrame()
percentage_change_df = df
# Convert 'date' to datetime format if it's not already
percentage_change_df['date'] = pd.to_datetime(percentage_change_df['date'])

# Extract the year from the 'date' column and create a new column called 'year'
percentage_change_df['year'] = percentage_change_df['date'].dt.year
# create a column "month" to match with CPI data
percentage_change_df['month'] = percentage_change_df['date'].dt.month

#print(df.columns)
#print(df['price'].dtype)

# Convert 'price' to numeric value format if it's not already
percentage_change_df['price'] = pd.to_numeric(percentage_change_df['price'], errors='coerce')
# Group by year, commodity, and market to calculate the mean price

annual_prices = df.groupby(['year', 'commodity', 'market']).agg({'price': 'mean'}).reset_index()

# Rename the price column in annual_prices to something like 'annual_avg_price'
annual_prices.rename(columns={'price': 'annual_avg_price'}, inplace=True)

# Merge the annual_prices DataFrame back into the main df
percentage_change_df = pd.merge(percentage_change_df, annual_prices, how='left', on=['year', 'commodity', 'market'], suffixes=('', '_to_drop'))
percentage_change_df = percentage_change_df.drop(columns=[col for col in df.columns if col.endswith('_to_drop')])
percentage_change_df['percentage_change'] = ((percentage_change_df['price'] - percentage_change_df['annual_avg_price']) / percentage_change_df['annual_avg_price']) * 100
# Calculate the percentage change between the actual price and the annual average price
insert_Dataframe_To_DB(percentage_change_df, 'WFP_PRICE_DATA_PROCESSED')

Data successfully inserted into WFP_PRICE_DATA_PROCESSED.


## Calculating the real price, nominal index and real index for food prices tab

In [35]:
#Import the CPI data to the data frame and merge CPI data to the main dataframe
cpi_df = pd.DataFrame()
with engine.connect() as connection:
    query = text('''SELECT value, CPI_month, CPI_year FROM NEPAL_DASHBOARD_CPI''')
    result = connection.execute(query)
    print(query)  
    
    # Convert the result to a DataFrame
    cpi_df = pd.DataFrame(result.fetchall(), columns=result.keys())

# Print the DataFrame
print(cpi_df.columns)



SELECT value, CPI_month, CPI_year FROM NEPAL_DASHBOARD_CPI
Index(['value', 'CPI_month', 'CPI_year'], dtype='object')


In [36]:
percentage_change_CPI_df = pd.merge(percentage_change_df, cpi_df, left_on=['year', 'month'], right_on=['CPI_year', 'CPI_month'], how='left', suffixes=('', '_to_drop'))
percentage_change_CPI_df = percentage_change_CPI_df.drop(columns=[col for col in df.columns if col.endswith('_to_drop')])

#change the column name of 'value' to CPI
percentage_change_CPI_df = percentage_change_CPI_df.rename(columns={'value':'CPI'})
insert_Dataframe_To_DB(percentage_change_CPI_df, 'WFP_PRICE_DATA_PROCESSED')

Data successfully inserted into WFP_PRICE_DATA_PROCESSED.


## calculate the real price and nominal index and real index

In [37]:
#calculate the real price and nominal index and real index
# real price = nominal (price/CPI)*100
percentage_change_CPI_real_df = pd.DataFrame()
percentage_change_CPI_real_df = percentage_change_CPI_df

if pd.api.types.is_numeric_dtype(percentage_change_CPI_real_df["CPI"]):
    percentage_change_CPI_real_df["real_price"] = (percentage_change_CPI_real_df["price"]/percentage_change_CPI_real_df["CPI"])*100

insert_Dataframe_To_DB(percentage_change_CPI_real_df, 'WFP_PRICE_DATA_PROCESSED')



Data successfully inserted into WFP_PRICE_DATA_PROCESSED.


In [50]:


# Group by 'commodity' and 'market', then get the 'real_price' for January 2020 --> there are missing values
# so, it would be good to get the average between Jan, Feb, and March
# even though, there is no real price, then get the first real price from the series.
# Group by 'commodity' and 'market', then get the average 'real_price' for January, February, and March of 2020
jan_feb_mar_2020_real_prices = pd.DataFrame()

jan_feb_mar_2020_real_prices = percentage_change_CPI_real_df[(percentage_change_CPI_real_df['year'] == 2020) & (percentage_change_CPI_real_df['month'].isin([1, 2, 3]))]
missing_real_prices =  percentage_change_CPI_real_df[~((percentage_change_CPI_real_df['year'] == 2020) & (percentage_change_CPI_real_df['month'].isin([1, 2, 3])))]
missing_real_prices_2020_after = missing_real_prices[percentage_change_CPI_real_df['year'] >= 2020]
# get the latest datapoint after grouping by groupby(['admin1', 'admin2','market', 'commodity' ])
missing_real_prices_2020_after_sort = missing_real_prices_2020_after.sort_values(['admin1', 'admin2','market', 'commodity' ], ascending=[True, True, True, True])
missing_real_prices_2020_after_sort.reset_index(drop=True)

# Get the earliest values for each series
earliest_real_values = missing_real_prices_2020_after_sort.groupby(['admin1', 'admin2','market', 'commodity' ], as_index=False).first()
earliest_real_values.rename(columns={'real_price': 'real_price_reference'}, inplace=True)

insert_Dataframe_To_DB(earliest_real_values, 'WFP_PRICE_DATA_PROCESSED_MISSING REAL_REFERENCE')

# Calculate the average 'real_price' for each group ('commodity' and 'market')
#avg_real_prices_grouped = jan_feb_mar_2020_real_prices.groupby(['admin1', 'admin2','market', 'commodity' ],as_index=False)['real_price'].mean().reset_index()

avg_real_prices_grouped = jan_feb_mar_2020_real_prices.groupby(['admin1', 'admin2', 'market', 'commodity'], as_index=False).agg({'real_price': 'mean'})
# Renaming the 'real_price' column to 'nominal_pricereference'
avg_real_prices_grouped.rename(columns={'real_price': 'real_price_reference'}, inplace=True)

 
# Merge this 'real_reference_price' from 2020 Jan  back into the main dataframe
percentage_change_CPI_real_index_df = pd.merge(percentage_change_CPI_real_df, avg_real_prices_grouped, on=['admin1', 'admin2','commodity', 'market'], how='left',suffixes=('', '_to_drop'))
percentage_change_CPI_real_index_df = percentage_change_CPI_real_index_df.drop(columns =[col for col in percentage_change_CPI_real_df.columns if col.endswith('_to_drop')])

insert_Dataframe_To_DB(percentage_change_CPI_real_index_df, 'WFP_PRICE_DATA_PROCESSED_REAL_REFERENCE1')

''' CHECK THE EARLIST REAL VALUES AND ADD THEM TO THE MAIN DF IF THE MAIN DF DOESN'T HAVE THE REAL REFERENCE PRICES '''

percentage_change_CPI_real_index_early_df = pd.DataFrame()
# Merge the dataframes
percentage_change_CPI_real_index_early_df = pd.merge(
    percentage_change_CPI_real_index_df, 
    earliest_real_values[['admin1', 'admin2', 'commodity', 'market', 'real_price_reference']],  
    on=['admin1', 'admin2', 'commodity', 'market'], how='left', suffixes=('', '_to_drop'))



# Check if 'real_price_reference' is null in percentage_change_CPI_real_nominal_index_df
mask = percentage_change_CPI_real_index_early_df['real_price_reference'].isnull()

# Fill missing values in percentage_change_CPI_real_nominal_index_df['real_price_reference'] from avg_real_prices_grouped['real_price_reference']
percentage_change_CPI_real_index_early_df.loc[mask, 'real_price_reference'] = percentage_change_CPI_real_index_early_df.loc[mask, 'real_price_reference_to_drop']


percentage_change_CPI_real_index_early_df = percentage_change_CPI_real_index_early_df.drop(columns =[col for col in percentage_change_CPI_real_index_df.columns if col.endswith('_to_drop')])


insert_Dataframe_To_DB(percentage_change_CPI_real_index_early_df, 'WFP_PRICE_DATA_PROCESSED_REAL_REFERENCE2')



'''
# Calculate the index based on the real_price_reference for each group, only if nominal_price_reference is not NaN
df_real_price_reference['real_price_index'] = np.where(
    df_real_price_reference['real_price_reference'].notna(),
    (df_real_price_reference['real_price'] / df_real_price_reference['real_price_reference']) * 100,
    np.nan
)
'''




/tmp/ipykernel_104/2402028871.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  missing_real_prices_2020_after = missing_real_prices[percentage_change_CPI_real_df['year'] >= 2020]


Data successfully inserted into WFP_PRICE_DATA_PROCESSED_MISSING REAL_REFERENCE.
Data successfully inserted into WFP_PRICE_DATA_PROCESSED_REAL_REFERENCE1.
Data successfully inserted into WFP_PRICE_DATA_PROCESSED_REAL_REFERENCE2.


"\n# Calculate the index based on the real_price_reference for each group, only if nominal_price_reference is not NaN\ndf_real_price_reference['real_price_index'] = np.where(\n    df_real_price_reference['real_price_reference'].notna(),\n    (df_real_price_reference['real_price'] / df_real_price_reference['real_price_reference']) * 100,\n    np.nan\n)\n"

In [44]:

# Group by 'commodity' and 'market', then get the 'real_price' for January 2020 --> there are missing values
# so, it would be good to get the average between Jan, Feb, and March

# Group by 'commodity' and 'market', then get the average 'real_price' for January, February, and March of 2020
jan_feb_mar_2020_prices = merged_df[(merged_df['year'] == 2020) & (merged_df['month'].isin([1, 2, 3]))]
print(jan_feb_mar_2020_prices.head())

jan_feb_mar_2020_prices.to_csv("jan_feb_mar_2020_prices.csv")
files.download("jan_feb_mar_2020_prices.csv")


# Calculate the average 'real_price' for each group ('commodity' and 'market')
avg_prices_grouped = jan_feb_mar_2020_prices.groupby(['admin1', 'admin2','market', 'commodity' ])['price'].mean().reset_index()
# Renaming the 'price' column to 'nominal_pricereference'
avg_prices_grouped.rename(columns={'price': 'nominal_price_reference'}, inplace=True)
avg_prices_grouped.to_csv("avg_prices_grouped.csv")
files.download("avg_prices_grouped.csv")


NameError: name 'merged_df' is not defined

In [ ]:

# Merge this 'base_price' back into the main dataframe
df_nominal_price_reference = pd.merge(merged_df, avg_prices_grouped, on=['admin1', 'admin2','commodity', 'market'], how='left')
# Rename the price column

df_nominal_price_reference.to_csv("df_nominal_price_reference.csv")
files.download("df_nominal_price_reference.csv")

print(df_nominal_price_reference.head)


# Calculate the index based on the nominal_price_reference for each group, only if nominal_price_reference is not NaN
df_nominal_price_reference['nominal_price_index'] = np.where(
    df_nominal_price_reference['nominal_price_reference'].notna(),
    (df_nominal_price_reference['price'] / df_nominal_price_reference['nominal_price_reference']) * 100,
    np.nan
)

df_nominal_price_reference.to_csv("df_nominal_price_index.csv")
files.download("df_nominal_price_index.csv")


In [ ]:
df_final_output = pd.merge(df_nominal_price_reference, df_real_price_reference, on= ['admin1', 'admin2','commodity', 'market', 'date'], how ='left' )
print(df_final_output.columns)

df_final_output.drop(['latitude_y', 'longitude_y', 'category_y',
       'unit_y', 'priceflag_y', 'pricetype_y', 'currency_y', 'price_y',
       'usdprice_y', 'year_y', 'month_y', 'annual_avg_price_y',
       'percentage_change_y', 'CPI_y', 'CPI_year_y', 'CPI_month_y',
       'real_price_y', ], axis=1, inplace=True)

# rename
df_final_output.rename(columns={'latitude_x': 'latitude', 'longitude_x': 'longitude','category_x': 'category','unit_x': 'unit',
                                'priceflag_x': 'priceflag', 'pricetype_x': 'pricetype','currency_x': 'currency','price_x': 'price',
                                'usdprice_x': 'usdprice', 'year_x': 'year','month_x': 'month','annual_avg_price_x': 'annual_avg_price',
                                'percentage_change_x': 'percentage_change', 'CPI_x': 'CPI','real_price_x': 'real_price'
                                }, inplace=True)

df_final_output.to_csv("df_final_output.csv")
files.download("df_final_output.csv")